In [ ]:
### https://realpython.com/caching-external-api-requests/

In [169]:
import time
import requests
import pandas as pd
import json

In [8]:
#### Define possible sources of classification
CFurl = "http://classyfire.wishartlab.com"
GNPSurl = 'https://gnps-classyfire.ucsd.edu'

In [44]:
validinchikey = 'ATUOYWHBWRKTHZ-UHFFFAOYSA-N' #Test case for a valid inchikey

In [3]:
return_format="json" 

In [183]:
#### Make a request to the site
r = requests.get('%s/entities/%s.%s' % (CFurl, validinchikey, return_format),
                     headers={"Content-Type": "application/%s" % return_format})
print(r) #200 if the inchikey is found
#print(r.text)

<Response [200]>


In [46]:
### Use the GNPS url
r = requests.get('%s/entities/%s.%s' % (GNPSurl, validinchikey, return_format),
                     headers={"Content-Type": "application/%s" % return_format})
print(r)
#print(r.text)

<Response [200]>


In [28]:
pd.json_normalize(json.loads(r.text)) #Convert output to DF

,smiles,inchikey,intermediate_nodes,alternative_parents,molecular_framework,substituents,description,external_descriptors,ancestors,predicted_chebi_terms,...,class.chemont_id,class.url,subclass.name,subclass.description,subclass.chemont_id,subclass.url,direct_parent.name,direct_parent.description,direct_parent.chemont_id,direct_parent.url
0,CCC,InChIKey=ATUOYWHBWRKTHZ-UHFFFAOYSA-N,[],[],Aliphatic acyclic compounds,"[Acyclic alkane, Alkane, Aliphatic acyclic com...",This compound belongs to the class of organic ...,"[{'source': 'CHEBI', 'source_id': 'CHEBI:32879...","[Alkanes, Chemical entities, Hydrocarbons, Org...","[alkane (CHEBI:18310), chemical entity (CHEBI:...",...,CHEMONTID:0004474,http://classyfire.wishartlab.com/tax_nodes/C00...,Alkanes,Acyclic branched or unbranched hydrocarbons ha...,CHEMONTID:0002500,http://classyfire.wishartlab.com/tax_nodes/C00...,Alkanes,Acyclic branched or unbranched hydrocarbons ha...,CHEMONTID:0002500,http://classyfire.wishartlab.com/tax_nodes/C00...


## Write functions

In [189]:
def requestCF(inchikey,url=CFurl):
    '''
    Input: takes an inchikey
    Optional: one of two URLs, Wishart lab (default, recommended) or GNPS
    '''
    resp = requests.get('%s/entities/%s.%s' % (url, inchikey, 'json'))
    if resp.status_code == 200 and len(resp.text) > 2:
        return resp.text
    ## If not, None gets returned

#### Test

Some inchikeys are classified in the Wishart lab but not in GNPS

In [196]:
requestCF(validinchikey,url=GNPSurl)

'{"smiles": "CCC", "inchikey": "InChIKey=ATUOYWHBWRKTHZ-UHFFFAOYSA-N", "kingdom": {"name": "Organic compounds", "description": "Compounds that contain at least one carbon atom, excluding isocyanide/cyanide and their non-hydrocarbyl derivatives, thiophosgene, carbon diselenide, carbon monosulfide, carbon disulfide, carbon subsulfide, carbon monoxide, carbon dioxide, Carbon suboxide, and dicarbon monoxide.", "chemont_id": "CHEMONTID:0000000", "url": "http://classyfire.wishartlab.com/tax_nodes/C0000000"}, "superclass": {"name": "Hydrocarbons", "description": "Organic compounds made up only of carbon and hydrogen atoms.", "chemont_id": "CHEMONTID:0002837", "url": "http://classyfire.wishartlab.com/tax_nodes/C0002837"}, "class": {"name": "Saturated hydrocarbons", "description": "Hydrocarbons that contains only saturated carbon atoms, which are linked to one another through single bonds. These includes alkanes and cycloalkanes.", "chemont_id": "CHEMONTID:0004474", "url": "http://classyfire.wi

In [197]:
requestCF(validinchikey,url=CFurl)

'{"smiles":"CCC","inchikey":"InChIKey=ATUOYWHBWRKTHZ-UHFFFAOYSA-N","kingdom":{"name":"Organic compounds","description":"Compounds that contain at least one carbon atom, excluding isocyanide/cyanide and their non-hydrocarbyl derivatives, thiophosgene, carbon diselenide, carbon monosulfide, carbon disulfide, carbon subsulfide, carbon monoxide, carbon dioxide, Carbon suboxide, and dicarbon monoxide.","chemont_id":"CHEMONTID:0000000","url":"http://classyfire.wishartlab.com/tax_nodes/C0000000"},"superclass":{"name":"Hydrocarbons","description":"Organic compounds made up only of carbon and hydrogen atoms.","chemont_id":"CHEMONTID:0002837","url":"http://classyfire.wishartlab.com/tax_nodes/C0002837"},"class":{"name":"Saturated hydrocarbons","description":"Hydrocarbons that contains only saturated carbon atoms, which are linked to one another through single bonds. These includes alkanes and cycloalkanes.","chemont_id":"CHEMONTID:0004474","url":"http://classyfire.wishartlab.com/tax_nodes/C000447

#### Edge cases

In [198]:
cfinchikey = 'GNGSNCPQCZJNBI-VAYFVJFHSA-N' 
# Empty in GNPS but ok in Wishart lab
#https://gnps-classyfire.ucsd.edu/entities/GNGSNCPQCZJNBI-VAYFVJFHSA-N.json
#http://classyfire.wishartlab.com/entities/GNGSNCPQCZJNBI-VAYFVJFHSA-N.json

In [199]:
requestCF(cfinchikey,url=GNPSurl) #None gets returned

In [200]:
requestCF(cfinchikey,url=CFurl) #

'{"smiles":"COC(=O)CC[C@@H](C)[C@H]1C[C@@H](O)[C@H]2[C@@H]3[C@H](O)C[C@@H]4C[C@H](O)CC[C@]4(C)[C@H]3CC[C@]12C","inchikey":"InChIKey=GNGSNCPQCZJNBI-VAYFVJFHSA-N","kingdom":{"name":"Organic compounds","description":"Compounds that contain at least one carbon atom, excluding isocyanide/cyanide and their non-hydrocarbyl derivatives, thiophosgene, carbon diselenide, carbon monosulfide, carbon disulfide, carbon subsulfide, carbon monoxide, carbon dioxide, Carbon suboxide, and dicarbon monoxide.","chemont_id":"CHEMONTID:0000000","url":"http://classyfire.wishartlab.com/tax_nodes/C0000000"},"superclass":{"name":"Lipids and lipid-like molecules","description":"Fatty acids and their derivatives, and substances related biosynthetically or functionally to these compounds.","chemont_id":"CHEMONTID:0000012","url":"http://classyfire.wishartlab.com/tax_nodes/C0000012"},"class":{"name":"Steroids and steroid derivatives","description":"Compounds based on the cyclopenta[a]phenanthrene carbon skeleton, par

In [202]:
stereo='ZTQNGKJCDYUACE-OAHLLOKONA-N'
nostereo = 'ZTQNGKJCDYUACE-UHFFFAOYSA-N' 
# Some keys are classified only with the second block changed

In [204]:
requestCF(stereo,url=GNPSurl) or requestCF(stereo,url=CFurl) #None gets returned#None gets returned

In [207]:
requestCF(nostereo,url=GNPSurl) #Second block of inchikey is different, no response in GNPS

In [208]:
requestCF(nostereo,url=CFurl) #Classified in Wishart lab's database

'{"smiles":"COC1=CC=C(CC2C3=CC(O)=C(OC)C=C3CC[N]2(C)[O-])C=C1O","inchikey":"InChIKey=ZTQNGKJCDYUACE-UHFFFAOYSA-N","kingdom":{"name":"Organic compounds","description":"Compounds that contain at least one carbon atom, excluding isocyanide/cyanide and their non-hydrocarbyl derivatives, thiophosgene, carbon diselenide, carbon monosulfide, carbon disulfide, carbon subsulfide, carbon monoxide, carbon dioxide, Carbon suboxide, and dicarbon monoxide.","chemont_id":"CHEMONTID:0000000","url":"http://classyfire.wishartlab.com/tax_nodes/C0000000"},"superclass":{"name":"Organoheterocyclic compounds","description":"Compounds containing a ring with least one carbon atom and one non-carbon atom.","chemont_id":"CHEMONTID:0000002","url":"http://classyfire.wishartlab.com/tax_nodes/C0000002"},"class":{"name":"Isoquinolines and derivatives","description":"Aromatic polycyclic compounds containing an isoquinoline moiety, which consists of a benzene ring fused to a pyridine ring and forming benzo[c]pyridine."

# Caching requests 

In [72]:
import requests_cache

In [73]:
!ls 

PyViz.ipynb                 pathosMultiprocessing.ipynb
chunk.ipynb                 python_and_xargs.ipynb
decorators.ipynb            requests_and_cache.ipynb
errorHandling.ipynb         sns_custom_colors.ipynb
ggplot_on_python.ipynb      temp-plot.html
multiprocessing.ipynb       test_xargs.sh


In [110]:
requests_cache.install_cache('classyfire_cache')

In [109]:
!open .

In [78]:
resp = requests.get('%s/entities/%s.%s' % (url, inchikey, return_format),
                     headers={"Content-Type": "application/%s" % return_format})

In [82]:
s=time.time()
for i in range(10):
    requests.get('http://httpbin.org/delay/1')
e=time.time()    
print('{}s'.format(round(e-s,2)))

1.27s


In [83]:
s=time.time()
for i in range(10):
    requests.get('http://httpbin.org/delay/1')
e=time.time()    
print('{}s'.format(round(e-s,2)))

0.05s


In [104]:
#### List of inchikeys that had previously failed
listIKs = [
    'JTTGPCRBDUOCGG-DCYUTMKDSA-N',
    'GNGSNCPQCZJNBI-VAYFVJFHSA-N',
    'QNRIYEYAHVEGQJ-SWQVIYSDSA-N',
    'FTOTVNJFGHCOCZ-ZOXBSTLDSA-N',
    'MEYVIHRQCGHFDC-AKTDRNHWSA-N',
    'SJBHFAAZLAILHS-XFOSWZMQNA-N',
    'MWTIGLPPQBNUFP-RRHRGVEJNA-N',
    'UXWZMQPNSYWAHX-VQUKNVSGSA-N',
    'MGFJVMMIYCGBBF-DOLKRJOSNA-N',
    'ZTQNGKJCDYUACE-OAHLLOKONA-N',
    'QSJOEQXFYCTBSY-HDXKHDETNA-N',
    'GKFNHDBVWFEZCJ-LQFNOIFHNA-N',
    'JAKXTPZLQBSVBB-AEDMQFBUNA-N',
    'OMFHIZTYMHFNIE-UHFFFAOYNA-N',
    'LRPBAJOIJTWGMK-KDIZOZADNA-N',
    'SVWBXNAUENUONE-LDLOPFEMNA-N',
    'NEUKLKKOFRWHDB-AUCIZUAGSA-N',
    'ZDRGKGZHXIWXJD-QJEXQQAGSA-N',
    'XFFRLQGZYVVLQI-FAIXQHPJSA-N',
    'FKJNKIIHHYXTFQ-SUNRTHERSA-N',
    'JLZYFORTRUGDQW-YVCFFHMXSA-N',
    'GBRCKWHNYQTJKH-NXHCNWFISA-N',
    'XZOUIUJOUOGCSM-OLSVMMKASA-N']

In [167]:
### Run this chunk without pre-installing the cache
outReqsNew = dict()
failed = list()
s=time.time()
for i in listIKs:
    r = requestCF(i)
    if r is not None:
        outReqsNew[i] = r
    else:
        failed.append(i)
e=time.time()    
print('\n {} elements in {}s'.format(len(outReqsNew),round(e-s,2)))


 9 elements in 2.34s


In [168]:
failed

['JTTGPCRBDUOCGG-DCYUTMKDSA-N',
 'FTOTVNJFGHCOCZ-ZOXBSTLDSA-N',
 'MEYVIHRQCGHFDC-AKTDRNHWSA-N',
 'SJBHFAAZLAILHS-XFOSWZMQNA-N',
 'MWTIGLPPQBNUFP-RRHRGVEJNA-N',
 'UXWZMQPNSYWAHX-VQUKNVSGSA-N',
 'MGFJVMMIYCGBBF-DOLKRJOSNA-N',
 'ZTQNGKJCDYUACE-OAHLLOKONA-N',
 'QSJOEQXFYCTBSY-HDXKHDETNA-N',
 'GKFNHDBVWFEZCJ-LQFNOIFHNA-N',
 'JAKXTPZLQBSVBB-AEDMQFBUNA-N',
 'OMFHIZTYMHFNIE-UHFFFAOYNA-N',
 'LRPBAJOIJTWGMK-KDIZOZADNA-N',
 'SVWBXNAUENUONE-LDLOPFEMNA-N']

Try again, the second time around it must be faster since the ones already visited are stored 

In [ ]:
requests_cache.install_cache('classyfire_cache')

In [127]:
%time
outReqs1 = dict()
s=time.time()
for i in listIKs[0:10]:
    outReqs[i] = requestCF(i)
e=time.time()    
print('{}s'.format(round(e-s,2)))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
1.19s


In [128]:
%time
outReqs2 = dict()
s=time.time()
for i in listIKs[0:10]:
    outReqs[i] = requestCF(i)
e=time.time()    
print('{}s'.format(round(e-s,2)))

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.82 µs
1.17s


In [130]:
outReqsNew == outReqs1 == outReqs2

True

In [132]:
'{}/entities/{}.{}'.format(url, inchikey, return_format)

'http://classyfire.wishartlab.com/entities/GNGSNCPQCZJNBI-VAYFVJFHSA-N.json'

In [ ]:
requests.get('%s/entities/%s.%s' % (url, inchikey, return_format),
                     headers={"Content-Type": "application/%s" % return_format})

In [134]:
r = requests.get('{}/entities/{}.{}'.format(url, inchikey, return_format))

In [136]:
r.from_cache

True

In [139]:
r = requests.get('{}/entities/{}.{}'.format(url, 'ZTQNGKJCDYUACE-UHFFFAOYSA-N', return_format))

In [140]:
r.from_cache

False

### Try, except

In [220]:
def get_cf(inchikey,url,verbose=False):
    '''
    Modified from https://gitlab.unige.ch/Pierre-Marie.Allard/pybatchclassyfire/-/blob/master/pybatchclassyfire/pybatchclassyfire.py
    Adds the original inchikey used to keep track of the molecules annotated.
    '''
    if verbose:
        print(inchikey)
    entity = None
    try:
        entity = requestCF(inchikey,url)
    except KeyboardInterrupt:
        raise
    if entity is None:
        except
            new_inchi_no_stereo = inchikey.split("-")[0] + "-UHFFFAOYSA-N"
        try:
            entity = requestCF(new_inchi_no_stereo,url)
        except KeyboardInterrupt:
            raise
        except:
            return entity
    ### Added chunk: Returns the original inchikey used to keep track in case the second block was changed
#     if entity != None:
#         entity['original'] = inchikey
    return entity


SyntaxError: invalid syntax (<ipython-input-220-afa8b9d1f3c6>, line 14)

In [246]:
def requestCF(inchikey,url=CFurl):
    '''
    Input: takes an inchikey
    Optional: one of two URLs, Wishart lab (default, recommended) or GNPS
    '''
    resp = requests.get('%s/entities/%s.%s' % (url, inchikey, 'json'))
    return resp
    ## If not, None gets returned

In [249]:
try:
    r = requestCF('ZTQNGKJCDYUACE-OAHLLOKONA-N',CFurl)
    r.raise_for_status()
except requests.exceptions.HTTPError as err:
    print('not valid')

not valid


In [451]:
def getCF(inchikey): 
    '''
    Handle exceptions
    '''
    new_inchi_no_stereo = inchikey.split("-")[0] + "-UHFFFAOYSA-N"
    try:
        #print('CF original')
        r = requestCF(inchikey,CFurl)
        r.raise_for_status()
    except requests.exceptions.HTTPError as err:
        try:
            #print('CF no stereo')            
            r = requestCF(new_inchi_no_stereo,CFurl)
            r.raise_for_status()
        except requests.exceptions.HTTPError as err:
            try:
                #print('GNPS original')
                r = requestCF(inchikey,GNPSurl)
                r.raise_for_status()
            except requests.exceptions.HTTPError as err:
                try:
                    #print('GNPS no stereo')            
                    r = requestCF(new_inchi_no_stereo,GNPSurl)
                    r.raise_for_status()
                except requests.exceptions.HTTPError as err:
                    r = None
    except Exception: #Catch all
        raise
    return r
#     except:
#         try:
#             r = requestCF(inchikey,GNPSurl)
#             r.raise_for_status()
#         except requests.exceptions.HTTPError as err:
#             raise


In [437]:
def wrapResponses(inchikey):
    '''
    '''
    cf = getCF(inchikey)
    if cf is not None:
        if len(cf.text) > 2:
            out = json.loads(cf.text)
            out['original'] = inchikey
            return out

In [458]:
### Without cache
s = time.time()
getInchiKeys = list(map(wrapResponses,listIKs))
e = time.time()
print('{} in {}s'.format(len(getInchiKeys),round(e-s,2)))

23 in 5.72s


In [449]:
### With cache
s = time.time()
getInchiKeys = list(map(wrapResponses,listIKs))
e = time.time()
print('{} in {}s'.format(len(getInchiKeys),round(e-s,2)))

23 in 3.04s


In [443]:
classifiedIKs = list(filter(None, getInchiKeys))

In [445]:
test = requestCF('GNGSNCPQCZJNBI-VAYFVJFHSA-N',url=CFurl)

,smiles,inchikey,kingdom,superclass,class,subclass,intermediate_nodes,direct_parent,alternative_parents,molecular_framework,substituents,description,external_descriptors,ancestors,predicted_chebi_terms,predicted_lipidmaps_terms,classification_version,original
0,COC(=O)CC[C@@H](C)[C@H]1C[C@@H](O)[C@H]2[C@@H]...,InChIKey=GNGSNCPQCZJNBI-VAYFVJFHSA-N,"{'name': 'Organic compounds', 'description': '...","{'name': 'Lipids and lipid-like molecules', 'd...","{'name': 'Steroids and steroid derivatives', '...","{'name': 'Bile acids, alcohols and derivatives...","[{'name': 'Hydroxy bile acids, alcohols and de...","{'name': 'Trihydroxy bile acids, alcohols and ...","[{'name': '7-hydroxysteroids', 'description': ...",Aliphatic homopolycyclic compounds,"[Trihydroxy bile acid, alcohol, or derivatives...",This compound belongs to the class of organic ...,[],"[3-alpha-hydroxysteroids, 3-hydroxysteroids, 7...","[7-hydroxy steroid (CHEBI:36844), 3alpha-hydro...","[Sterol Lipids (ST), Fatty esters (FA07), Bile...",2.1,GNGSNCPQCZJNBI-VAYFVJFHSA-N
1,C[C@H](CCC(=O)NCCS(O)(=O)=O)[C@H]1CC[C@H]2[C@H...,InChIKey=QNRIYEYAHVEGQJ-SWQVIYSDSA-N,"{'name': 'Organic compounds', 'description': '...","{'name': 'Lipids and lipid-like molecules', 'd...","{'name': 'Steroids and steroid derivatives', '...","{'name': 'Bile acids, alcohols and derivatives...",[],{'name': 'Taurinated bile acids and derivative...,"[{'name': 'Monohydroxy bile acids, alcohols an...",Aliphatic homopolycyclic compounds,"[Taurinated bile acid, Monohydroxy bile acid, ...",This compound belongs to the class of organic ...,[],"[3-alpha-hydroxysteroids, 3-hydroxysteroids, 7...","[bile acid (CHEBI:3098), 7-oxo steroid (CHEBI:...","[Bile acids and derivatives (ST04), Sterol Lip...",2.1,QNRIYEYAHVEGQJ-SWQVIYSDSA-N
2,CC(CCC(=O)NC(CCCCN)C(O)=O)C1CCC2C3C(O)CC4CC(O)...,InChIKey=FTOTVNJFGHCOCZ-UHFFFAOYSA-N,"{'name': 'Organic compounds', 'description': '...","{'name': 'Lipids and lipid-like molecules', 'd...","{'name': 'Steroids and steroid derivatives', '...","{'name': 'Bile acids, alcohols and derivatives...",[],{'name': 'Glycinated bile acids and derivative...,"[{'name': 'Trihydroxy bile acids, alcohols and...",Aliphatic homopolycyclic compounds,"[Glycinated bile acid, Trihydroxy bile acid, a...",This compound belongs to the class of organic ...,[],"[12-hydroxysteroids, 3-hydroxysteroids, 7-hydr...","[bile acid (CHEBI:3098), 7-hydroxy steroid (CH...","[Bile acids and derivatives (ST04), Sterol Lip...",2.1,FTOTVNJFGHCOCZ-ZOXBSTLDSA-N
3,CC(CCCNCC(O)=O)C1CCC2C3CCC4CC(O)CCC4(C)C3CC(O)...,InChIKey=SJBHFAAZLAILHS-UHFFFAOYSA-N,"{'name': 'Organic compounds', 'description': '...","{'name': 'Lipids and lipid-like molecules', 'd...","{'name': 'Steroids and steroid derivatives', '...","{'name': 'Azasteroids and derivatives', 'descr...",[],"{'name': '25-azasteroids and derivatives', 'de...","[{'name': '3-hydroxysteroids', 'description': ...",Aliphatic homopolycyclic compounds,"[25-azasteroid, Hydroxysteroid, 12-hydroxyster...",This compound belongs to the class of organic ...,[],"[12-hydroxysteroids, 25-azasteroids and deriva...","[3-hydroxy steroid (CHEBI:36834), 12-hydroxy s...",[Sterol Lipids (ST)],2.1,SJBHFAAZLAILHS-XFOSWZMQNA-N
4,CCCCCCCCCCCCCCCCOCC(COP([O-])(=O)OCC[N+](C)(C)...,InChIKey=MWTIGLPPQBNUFP-UHFFFAOYSA-N,"{'name': 'Organic compounds', 'description': '...","{'name': 'Lipids and lipid-like molecules', 'd...","{'name': 'Glycerophospholipids', 'description'...","{'name': 'Glycerophosphocholines', 'descriptio...",[],"{'name': 'Dialkylglycerol-3-phosphocholines', ...","[{'name': 'Phosphocholines', 'description': 'C...",Aliphatic acyclic compounds,"[Dialkylglycero-3-phosphocholine, Phosphocholi...",This compound belongs to the class of organic ...,[],"[Alkyl phosphates, Amines, Chemical entities, ...","[phosphocholines (CHEBI:36700), glycerol ether...","[Dialkylglycerophosphocholines (GP0104), Glyce...",2.1,MWTIGLPPQBNUFP-RRHRGVEJNA-N
5,CC(CCC(=O)NC(CC1=CNC2=CC=CC=C12)C(O)=O)C1CCC2C...,InChIKey=UXWZMQPNSYWA